# Categorical Variables

Intakes:

    Indigenous land from FUNAI
        https://www.gov.br/funai/pt-br/atuacao/terras-indigenas/geoprocessamento-e-mapas
    Ecoregion from RESOLVE
        https://developers.google.com/earth-engine/datasets/catalog/RESOLVE_ECOREGIONS_2017
    Protected areas from CEM - USP (2020)
        https://centrodametropole.fflch.usp.br/pt-br/download-de-dados
    Soil type from DSMW
        https://www.fao.org/soils-portal/data-hub/soil-maps-and-databases/faounesco-soil-map-of-the-world/en/
    
Exports images with binary masks for protected areas and indigenous land, or byte values for ecoregion and soil type.

In [1]:
import ee
import geemap

# Authenticate to Earth Engine
try:
  ee.Initialize()
except Exception as e:
  ee.Authenticate()
  ee.Initialize(project='ee-ana-zonia')

from utils import export_image
from utils import map_image

roi = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes").geometry().dissolve()


In [29]:

'''
fc_address -> GEE path to the feature collection
column_name -> column that designates the value of the feature we want to extract (in this case, the ID)
newname -> band name for the new image
'''

# Soil categories are strings, which Image objects can't handle, and ecoregion categories are large integers.
# To transform the feature collection into an image, we convert the categories to bytes by creating a dictionary.

def categorical_convert(fc_address, column_name, newname):
    # import feature collection and crop it to region of interest
    fc = ee.FeatureCollection(fc_address).filterBounds(roi) \
                .map(lambda feature: feature.intersection(roi))
    # create array with unique categories in the feature collection
    unique_category = fc.select([column_name]).aggregate_array(column_name).distinct()
    # assign numbers starting from 1 to each category
    category_dict = ee.Dictionary.fromLists(unique_category, ee.List.sequence(1, unique_category.length()))
    # reassign original category values to new values from dictionary (1 to total number of categories)
    converted_fc = fc.remap(unique_category, category_dict.values(), column_name)
    # return image with new values in byte type to facilitate exporting
    return converted_fc.reduceToImage([column_name], ee.Reducer.first()).byte().rename(newname)

In [34]:
ecoregions = (ee.FeatureCollection("RESOLVE/ECOREGIONS/2017").filterBounds(roi)
                .map(lambda feature: feature.intersection(roi)))
ecoregions_img = ecoregions.reduceToImage(['ECO_ID'], ee.Reducer.first()).rename('ecoreg')

biome = ee.FeatureCollection("projects/ee-ana-zonia/assets/br_biomes")
biome_img = biome.reduceToImage(['id'], ee.Reducer.first()).rename('biome')

soil_img = categorical_convert("projects/ee-ana-zonia/assets/DSMW", "DOMSOI", 'soil')

indig_land = ee.FeatureCollection("projects/ee-ana-zonia/assets/indig_land").filterBounds(roi)
indig_land_img = ee.Image().paint(indig_land, 1).unmask(0).byte().rename("indig")

protec = ee.FeatureCollection("projects/ee-ana-zonia/assets/unc_2021").filterBounds(roi)
protec_img = ee.Image().paint(protec, 1).unmask(0).byte().rename("protec")

In [ ]:
categorical = ecoreg.addBands([indig, soil, protec, biomes])
export_image(land_use, f"land_use_{max_age}_years_one_hectare_1985_forested")